# Exercício

Basicamente, o que fazemos aqui é examinar um conjunto de dados com Dados do cliente de comércio eletrônico para o site e aplicativo móvel de uma empresa. Em seguida, queremos ver se podemos construir um modelo de regressão que prediga os gastos anuais do cliente com o produto da empresa.

A primeira coisa a fazer é iniciar uma sessão Spark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
# Use o Spark para ler o arquivo csv dos clientes de comércio eletrônico.
data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

In [5]:
# Imprima o esquema do DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [8]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [9]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Configurando DataFrame para Machine Learning

In [10]:
# Algumas coisas que precisamos fazer antes que o Spark possa aceitar os dados!
# Precisa ter a forma de duas colunas
# ("label","features")

# Import VectorAssembler
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [13]:
output = assembler.transform(data)

In [14]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [19]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [20]:
final_data = output.select("features",'Yearly Amount Spent')

In [17]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                346|
|   mean|  496.1900914720963|
| stddev|  78.36486159699743|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                154|
|   mean|  506.3327758438319|
| stddev|   81.2283391449996|
|    min| 302.18954780965197|
|    max|  712.3963268096637|
+-------+-------------------+



In [23]:
# Crie um objeto Modelo de Regressão Linear
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [24]:
# Ajuste o modelo aos dados e chame este modelo de lrModel
lrModel = lr.fit(train_data,)

In [25]:
# Imprime os coeficientes e intercepta para regressão linear
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.812385007769638,38.426564015986685,0.4809639626786554,61.811716771966054] Intercept: -1053.6548783852147


In [26]:
test_results = lrModel.evaluate(test_data)

In [27]:
# Resultados interessantes ....
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 10.673630335217695|
|-10.755727390294965|
| 11.468603677449096|
|  3.575979626482763|
| 0.7580177960101082|
| -3.239753540367701|
|  -5.28743427903953|
|-1.5463181084822963|
| -3.294154674869617|
| -7.051592690147231|
| -4.196798181664974|
|-0.8766723306306403|
|-1.1887406789430202|
|-13.317011771591979|
| -5.529609977604309|
|  6.294647878566025|
|   8.39040920289142|
|-4.1704603064591765|
|  0.354849176254163|
|  2.268182805238439|
+-------------------+
only showing top 20 rows



In [28]:
unlabeled_data = test_data.select('features')

In [29]:
predictions = lrModel.transform(unlabeled_data)

In [30]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 397.9667207374098|
|[30.3931845423455...|330.68459719348857|
|[30.7377203726281...| 450.3121385187808|
|[31.3091926408918...|429.14473821345086|
|[31.3895854806643...| 409.3115932639728|
|[31.4252268808548...| 534.0064721951296|
|[31.5257524169682...| 449.2530610889214|
|[31.5761319713222...| 542.7729020978106|
|[31.6253601348306...| 379.6310554317938|
|[31.7207699002873...| 545.8265261681702|
|[31.7656188210424...| 500.7508798172721|
|[31.8627411090001...| 557.1748135046773|
|[31.9120759292006...|388.72345698465074|
|[31.9365486184489...| 440.5163966669202|
|[31.9453957483445...| 662.5495339152562|
|[31.9480174211613...| 455.6262290143318|
|[31.9549038566348...|431.60747073703556|
|[31.9673209478824...|449.92030154611143|
|[32.0047530203648...| 463.3911319443753|
|[32.0444861274404...|445.96164638131154|
+--------------------+------------

In [31]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.807512064508103
MSE: 96.18729289547198
